In [1]:
import pandas as pd

In [2]:
Datos_completos=pd.read_csv("../data/Datos_completos.csv")

### Analisis de variable

In [3]:
Datos_completos.head()

,Airline,Source,Destination,Duration,Total stops,Price,Date
0,Air France,NYC,PAR,7h 20m,nonstop,"1,031 SAR",2022-02-01
1,Air France,NYC,PAR,7h 20m,nonstop,"1,031 SAR",2022-02-01
2,Air France,NYC,PAR,7h 20m,nonstop,"1,031 SAR",2022-02-01
3,Air France,NYC,PAR,7h 10m,nonstop,"1,238 SAR",2022-02-01
4,Air France,NYC,PAR,7h 20m,nonstop,"1,238 SAR",2022-02-01


In [4]:
Datos_completos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55363 entries, 0 to 55362
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Airline      55363 non-null  object
 1   Source       55363 non-null  object
 2   Destination  55363 non-null  object
 3   Duration     55363 non-null  object
 4   Total stops  55363 non-null  object
 5   Price        55363 non-null  object
 6   Date         55363 non-null  object
dtypes: object(7)
memory usage: 3.0+ MB


In [5]:
#verificamos si exixte algun campo nulo
Datos_completos.isnull().sum()

Airline        0
Source         0
Destination    0
Duration       0
Total stops    0
Price          0
Date           0
dtype: int64

In [6]:
#verificamos si exisiten datos duplicados
Datos_completos[Datos_completos.duplicated()]

,Airline,Source,Destination,Duration,Total stops,Price,Date
1,Air France,NYC,PAR,7h 20m,nonstop,"1,031 SAR",2022-02-01
2,Air France,NYC,PAR,7h 20m,nonstop,"1,031 SAR",2022-02-01
6,Air France,NYC,PAR,7h 30m,nonstop,"1,238 SAR",2022-02-01
18,"Finnair, Iberia",NYC,PAR,10h 55m,1 stop,"1,080 SAR",2022-02-01
20,"Finnair, Air France",NYC,PAR,10h 15m,1 stop,"1,193 SAR",2022-02-01
...,...,...,...,...,...,...,...
55300,Air France,SVO,RUH,16h 45m,2 stops,"2,704 SAR",2022-04-28
55301,Air France,SVO,RUH,16h 45m,2 stops,"2,704 SAR",2022-04-28
55341,Qatar Airways,SVO,RUH,8h 30m,1 stop,"2,047 SAR",2022-04-30
55350,Air France,SVO,RUH,16h 45m,2 stops,"2,704 SAR",2022-04-30


#### Modificamos las variables a types que sean mas manejables

In [7]:
#Eliminar las datos duplicados
Data= Datos_completos.drop_duplicates().copy()
#creamos un columna Market para determinar la ruta
Data['Market']=Data['Source']+"-"+Data['Destination']
#Convertir la columna Price de SAR "riyal saudi" a USD "dolares"
# convert price to numerical format in USD
def clean_price(price):
    price = price.str.replace(',','',regex=True)
    price = price.str.replace('SAR','',regex=True)
    price = price.str.strip()
    price = round(pd.to_numeric(price)*0.27,2)
    return price
Data['Price']=clean_price(Data['Price'])

In [8]:
# Reemplazar valores en la columna 'Total stops'
# 'nonstop '= 0, '1 stop '=1, '2 stops '=2, '3 stops '=3
Data['Total stops'] = Data['Total stops'].replace({'nonstop ': 0, '1 stop ': 1,'2 stops ':2, '3 stops ':3})

In [9]:
#convertimos la duracion que sta en horas a minutos
def clean_Duration(x):
    try:
        x=x.split()
        hora=x[0].split(sep = "h")
        minutos=x[1].split(sep = "m")
        cont=int(hora[0])*60 + int(minutos[0])
    except Exception as e:
        print(e)
    return cont
Data['Duration']=Data['Duration'].apply(lambda x: clean_Duration(x))

In [10]:
Data['Date']=pd.to_datetime(Data['Date'])

In [11]:
Data

,Airline,Source,Destination,Duration,Total stops,Price,Date,Market
0,Air France,NYC,PAR,440,0,278.37,2022-02-01,NYC-PAR
3,Air France,NYC,PAR,430,0,334.26,2022-02-01,NYC-PAR
4,Air France,NYC,PAR,440,0,334.26,2022-02-01,NYC-PAR
5,Air France,NYC,PAR,450,0,334.26,2022-02-01,NYC-PAR
7,Lufthansa,NYC,PAR,425,0,365.31,2022-02-01,NYC-PAR
...,...,...,...,...,...,...,...,...
55358,"Air France, Egypt Air",SVO,RUH,1270,3,518.40,2022-04-30,SVO-RUH
55359,"Finnair, Pegasus Airlines",SVO,RUH,1090,3,561.60,2022-04-30,SVO-RUH
55360,Multiple Airlines,SVO,RUH,1055,3,564.57,2022-04-30,SVO-RUH
55361,Multiple Airlines,SVO,RUH,985,3,707.40,2022-04-30,SVO-RUH


Analisis la cantidad de Aerolineas

In [12]:
cant_aero=len(Data['Airline'].unique().tolist())
print(f"la cantidad de aerolineas es: {cant_aero}")

la cantidad de aerolineas es: 839


In [13]:
D=Data.groupby(['Airline'])['Price'].count().reset_index()
D['Percent']=D['Price']/Data.shape[0]
DD=D.sort_values('Price', ascending=False)
DD.head(20)

,Airline,Price,Percent
208,Delta,4052,0.080557
352,KLM,2880,0.057256
434,Multiple Airlines,2534,0.050378
147,American Airlines,2495,0.049602
92,Air France,2037,0.040497
397,Lufthansa,1942,0.038608
67,Air Canada,1870,0.037177
716,United Airlines,1772,0.035229
171,British Airways,1518,0.030179
481,Qatar Airways,1381,0.027455


In [14]:
cant=DD['Price'][:20].sum()
percentaje=round(DD['Percent'][:20].sum(),3)
print(f"Al analizar a fondo se observa que 20 aerilinas tienen {cant} datos que equivalen un {percentaje} % de la totalidad de los datos")

Al analizar a fondo se observa que 20 aerilinas tienen 31211 datos que equivalen un 0.62 % de la totalidad de los datos


Al analizar los datos se decide solo usar las anteriores 20 aerolines para hacer el analisis

In [15]:
Aerolineas_name=DD[:20]['Airline'].tolist()

In [16]:
Datos=pd.DataFrame()
for name in Aerolineas_name:
    df=Data[Data['Airline']==name]
    Datos=pd.concat([Datos, df], ignore_index=True)

In [17]:
Datos['Departure_Day'] = Datos['Date'].dt.day
Datos['Departure_Month'] = Datos['Date'].dt.month
Datos['Departure_Year'] = Datos['Date'].dt.year

In [18]:
Datos=Datos[['Airline','Source','Destination','Duration','Total stops','Market','Date','Departure_Day','Departure_Month','Departure_Year','Price']]

AL FINAL se tranformo la data cambiando y modificando las varibleas para un mejor manejo

* Se opto por utilizar solo las 20 aerolinas con mas datos
* La variable Date se subdividio en tres variables 'Departure_Day','Departure_Month','Departure_Year'.
* la duracion esta en minutos
* Total stops esta en una categoria numerica
* Price se cambia a USD
* Market se crea con la finalidad de mostrar las rutas mas facilmente


In [19]:
Datos.to_csv("../data/Datos_Transformados.csv",index=False)